In [2]:
# Import standard libraries
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_context('talk', font_scale = 1.05)

### Load data

In [3]:
df = pd.read_csv("C:\\Users\\juhic\\OneDrive\\Desktop\\DubsTech Datathon\\data.csv")
df['collection_week'] = pd.to_datetime(df['collection_week'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742253 entries, 0 to 742252
Columns: 128 entries, hospital_pk to total_staffed_pediatric_icu_beds_7_day_sum
dtypes: bool(2), datetime64[ns](1), float64(87), int64(29), object(9)
memory usage: 714.9+ MB


### Load required columns (sum only)

In [4]:
df2 = pd.read_excel("eda.xlsx", sheet_name = "columns")
df3 = pd.read_excel("eda.xlsx", sheet_name = "base_columns")

columns = list(df3['Name']) + list(df2['sum_columns'])
df = df[columns]

### Handle nulls & -9999 data records

In [5]:
df.fillna(0, inplace = True)
df.replace(to_replace = -999999, value = 2, inplace = True)

### Group by at given granularity

#### 1. State

In [6]:
# drop select base columns
drop_cols = ['hospital_pk','ccn','hospital_name','address','city','zip','hospital_subtype','fips_code',
             'is_metro_micro','geocoded_hospital_address','hhs_ids','is_corrected']
df_temp = df.drop(columns = drop_cols)

# group at state-week
df_state = df_temp.groupby(by = ['collection_week','state'], as_index = False).sum()

# sort dataframe at week level
df_state.sort_values(['state', 'collection_week'], ascending=[True, True], inplace=True)
df_state.reset_index(drop = True, inplace = True)
df_state.head()

,collection_week,state,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,inpatient_beds_used_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,...,previous_day_admission_pediatric_covid_confirmed_7_day_sum,previous_day_total_ED_visits_7_day_sum,previous_day_covid_ED_visits_7_day_sum,previous_day_admission_influenza_confirmed_7_day_sum,previous_week_personnel_covid_vaccinated_doses_administered_7_day,total_personnel_covid_vaccinated_doses_none_7_day,total_personnel_covid_vaccinated_doses_one_7_day,total_personnel_covid_vaccinated_doses_all_7_day,previous_week_patients_covid_vaccinated_doses_one_7_day,previous_week_patients_covid_vaccinated_doses_all_7_day
0,2020-03-20,AK,324.0,0.0,0.0,72.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-27,AK,1783.0,0.0,0.0,586.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-04-03,AK,9304.0,0.0,0.0,3486.0,0.0,24.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-04-10,AK,8667.0,0.0,0.0,2676.0,0.0,58.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-04-17,AK,7031.0,0.0,0.0,1840.0,0.0,167.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Select Features

In [7]:
req_cols = ['state',
            'collection_week',
            'inpatient_beds_used_covid_7_day_sum',
            'inpatient_beds_used_7_day_sum',
            'total_patients_hospitalized_confirmed_influenza_and_covid_7_day_sum',
            'all_pediatric_inpatient_bed_occupied_7_day_sum',
            'staffed_icu_pediatric_patients_confirmed_covid_7_day_sum',
            'previous_day_admission_adult_covid_confirmed_7_day_sum',
            'previous_day_covid_ED_visits_7_day_sum',
            'previous_day_admission_influenza_confirmed_7_day_sum',
            'previous_week_personnel_covid_vaccinated_doses_administered_7_day',
            'total_personnel_covid_vaccinated_doses_none_7_day',
            'previous_week_patients_covid_vaccinated_doses_all_7_day']

df_state = df_state[req_cols]

### Build Function to train Model

In [41]:
def VAR_model_train(df, state):
    
    # filter for specified state
    df = df.loc[df['state'] == state]
    df.drop(columns = 'state', inplace = True)
    df.set_index(keys = 'collection_week', inplace = True)
    
    # train-test split
    train = df[:-12]
    test = df[-12:]
    
    # build model & fit
    var_model = VARMAX(train, order = (4,0), enforce_stationarity = True)
    fitted_model = var_model.fit(disp = False)
    
    # forecast for 12 weeks
    n_forecast = 12
    predict = fitted_model.get_prediction(start = len(train), end = len(train)+n_forecast-1)
    predictions = predict.predicted_mean
    
    # store actuals vs predictions
    result = pd.DataFrame()
    result['week'] = test.index
    test.reset_index(drop = False, inplace = True)
    predictions.reset_index(drop = True, inplace = True)
    result['actual'] = test['inpatient_beds_used_covid_7_day_sum']
    result['predicted'] = predictions['inpatient_beds_used_covid_7_day_sum']
    
    return result

### Fit Model for every state

In [42]:
VAR_model_train(df_state, 'CA')

,week,actual,predicted
0,2022-11-25,28376.0,23115.107014
1,2022-12-02,33309.0,32957.329055
2,2022-12-09,34802.0,46134.817214
3,2022-12-16,34516.0,56371.888869
4,2022-12-23,33928.0,63867.368043
5,2022-12-30,35513.0,67595.594890
6,2023-01-06,32426.0,66114.171101
7,2023-01-13,27118.0,61325.331047
8,2023-01-20,22257.0,54539.158434
9,2023-01-27,20461.0,45383.498353


In [ ]:
states = df_state['state'].drop_duplicates()[0:2]
output = pd.DataFrame(columns = ['week', 'actual', 'predicted'])
for state in states:
    predicted = VAR_model_train(df_state, state)
    output.append(predicted, ignore_index = True)